# Implementing ML algorithms from scratch
## Part 1: Linear Regression, Locally Weighted Linear Regression, Logistic Regression


For the sake of simplicity, let us start with a simpler example of implementing a linear regression, which we could "upgrade" to logistic regression. That would help us understand the fundamentals needed later. *Please note that the bias term is omitted in a lot of models!* <br />
<h2>Linear Regression

We have a set of features (input variables) and a target output. This would be a single training example. We have a number of training examples, and our purpose is to come up with a function that takes features as an input and outputs something very close to the desired output. <br />

*Notation*: <br />
For a single training example:  $\textbf{x}_i \in \text{R}^m$ and $y_i \in \text{R}$ where $i = 1, ..., n$ <br />
$\textbf{w} \in \text{R}^m$ - vector of weights (to be learned)

We have a number of $n$ datapoints: $(\textbf{x}_i, y_i)$. Our purpose is minimize the error function (i.e. make our predictions as close as possible to target outputs): $E(\textbf{w}) = \frac{1}{2}\sum\limits_{k=1}^n (y_k - \textbf{w}^T\textbf{x}_k)^2$. More precisely, we need to come up with weights $\textbf{w}$ that minimize the given error function (the error function is squared error cost function, half is appended just for the sake of convenience). 

To do that, we will use a method called gradient descent. You can read more about it here: https://en.wikipedia.org/wiki/Gradient_descent

The main point of this method is to initialize weights to some random small values, then apply the following rule: <br />
$\textbf{w} \leftarrow \textbf{w} - \alpha \nabla E(\textbf{w})$, or <br />
$w_i \leftarrow w_i - \alpha \frac{dE}{dw_i}$ , where $\alpha$ is a so called learning rate.

Knowing that $E(\textbf{w}) = \frac{1}{2}\sum\limits_{k=1}^n (y_k - \textbf{w}^T\textbf{x}_k)^2$, it is easy to derive $\frac{dE}{dw_i}$, which is simply $\sum\limits_{k=1}^n (y_k - \textbf{w}^T\textbf{x}_k) (-x_{k,i})$ <br />
Hence the update rule is now simply: <br />
$w_i \leftarrow w_i + \alpha \sum\limits_{k=1}^n (y_k - \textbf{w}^T\textbf{x}_k) x_{k,i}$ <br />
$\Delta w_i \leftarrow \alpha \sum\limits_{k=1}^n (y_k - \textbf{w}^T\textbf{x}_k) x_{k,i}$ <br />
When do we stop updating? When either a new update is really small in absolute terms, or we have reached the maximum number of iterations. <br />
Note that the above update rule uses all the training examples for one single update (and that's why it's called a batch rule) <br />
Now let us implement this simple algorithm: 

In [24]:
import numpy as np
import random

# Suppose we are given N datapoints, then X is a (design) matrix NxM, y is a column vector Nx1. 
def linear_regression(X, y, max_number_iterations = 10000, learning_rate = 0.01, threshold = 0):
    iteration_numb = 0
    difference = 1
    N = X.shape[0]
    M = X.shape[1]
    
    # initialize w to random small numbers
    w = []
    for i in range(0, M):
        w.append(random.randint(1, 100) / 1000)
    w = np.asarray(w)

    while iteration_numb < max_number_iterations and difference > threshold:
        delta = np.zeros(M)
        # for every training example we have:
        for k in range(0, N):
            # calculate y_k - x_k*w
            error = y[k] - np.dot(X[k,:], w)
            for i in range(0, M):
                delta[i] = delta[i] + learning_rate * X[k, i] * error
        for i in range(0, M):
            w[i] = w[i] + delta[i];
            
        iteration_numb += 1
        difference = np.sqrt(delta.dot(delta))
    return w

# Just a simple test
# Note that the output y = 1*x1 + 0*x2 - 2*x3, so we expect the weights to be 1, 0 and -2
X = np.array([[5, 3, 2], [4, 2, 1], [3, 7, 0]])
y = np.array([1, 2, 3])
w = linear_regression(X, y)
print(w)

[ 1.00000000e+00  6.09469402e-11 -2.00000000e+00]


We can verify that we reached a correct set of weights by finding the global minimum weights using normal equations. <br />
For more details please refer to https://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)#Derivation_directly_in_terms_of_matrices <br />
In general, the solution is given by $\textbf{w} = (X^TX)^{-1}X^Ty$, where $X$ is a design matrix (i.e. the matrix, $i$-th row of which is the $i$-th training example, $\textbf{x}_i$) <br />
Note that sometimes it is not possible to derive an analytical solution (the case when the inverse does not exist)

In [25]:
from numpy.linalg import inv
# Check with analytical solution using normal equations
analytical_w = inv(X.transpose() @ X) @ X.transpose() @ y
print(analytical_w)

[ 1.0000000e+00 -8.8817842e-16 -2.0000000e+00]


We could see that our linear regression model is working well (both solutions are identical) <br />
Now that we have implemented a simple batch algorith, let us include extension for stochastic and mini-batch training. 

In [11]:
import numpy as np
import random
def lin_regression_advanced(X, y, batch_size = None, max_numb_iterations = 10000, learning_rate = 0.01, threshold = 0):
    iteration_numb = 0
    difference = 1
    N = X.shape[0]
    M = X.shape[1]
    if batch_size == None:
        # Default to batch training
        batch_size = N
    
    full_data = np.column_stack([X, y])
    
    # initialize w to random small numbers
    w = []
    for i in range(0, M):
        w.append(random.randint(1, 100) / 1000)
    w = np.asarray(w)

    while iteration_numb < max_numb_iterations and difference > threshold:
        np.random.shuffle(full_data)
        
        mini_batches = [
            full_data[k : k + batch_size]
            for k in range(0, N, batch_size)]
        
        for mini_batch in mini_batches:
            delta = np.zeros(M)
            
            for training_example in mini_batch:
                # training example is of shape (M+1), with M features (x) and 1 output (y)
                error = training_example[-1] - np.dot(training_example[:-1], w)
                for i in range(0, M):
                    delta[i] = delta[i] + learning_rate * training_example[i] * error
            
            #Once this mini-batch is over, update weights:
            for i in range(0, M):
                w[i] = w[i] + delta[i]
                
            # Optional: if delta change is really small (smaller than threshold), break from this loop,
            # return the current weights
            difference = np.sqrt(delta.dot(delta))
            if(difference < threshold):
                break
            
            
        iteration_numb += 1
    return w

# Just a simple test
X = np.array([[5, 3, 2], [4, 2, 1], [3, 7, 0]])
y = np.array([1, 2, 3])
w = lin_regression_advanced(X, y, batch_size = 10)
print(w)

[ 1.00000000e+00  5.93681512e-11 -2.00000000e+00]


Now that the simplest ML model is implemented, let us move on to a slightly more complicated one. 
<h2> Locally Weighted Linear Regression

This model is slightly more complicated, as compared to the ordinary linear regression, we weight each error. You could read more about it on the web. <br />
In mathematical terms, our objective now is to minimize the following cost function (I still denote the weights that should be *learned* as $\textbf{w}$, whereas the weights that are parameters of this model would be denoted as $a_i$ to avoid confusion): <br />
$E(\textbf{w}) = \frac{1}{2}\sum\limits_{k=1}^n a_k(y_k - \textbf{w}^T\textbf{x}_k)^2$ <br /> A fairly standard choice for $a_k$ is $e^{-\frac{(\textbf{x}_k - \textbf{x})^2}{2\tau^2}}$ where $\tau$ is a bandwidth parameter. <br />
The update rule is now simply: <br />
$w_i \leftarrow w_i + \alpha \sum\limits_{k=1}^n a_k(y_k - \textbf{w}^T\textbf{x}_k) x_{k,i}$ <br />
$\Delta w_i \leftarrow \alpha \sum\limits_{k=1}^n a_k(y_k - \textbf{w}^T\textbf{x}_k) x_{k,i}$ <br />
Note that the algorithm is non-parametric, so in order to predict an output for a new input, we train a new model. <br />
For normal equations part: <br />
$\textbf{w} = (X^TAX)^{-1}X^TAy$, where A is a diagonal matrix containing the weights $a_k$ on the main diagonal.

In [1]:
import numpy as np
import random
from numpy.linalg import inv

# Non-parametric regression, so we predict a value for a given input right away
def loc_weighted_regression(training_X, y, input_X, bandwidth = 0.5, max_number_iterations = 50000, 
                            learning_rate = 0.01, threshold = 0):
    iteration_numb = 0
    difference = 1
    N = training_X.shape[0]
    M = training_X.shape[1]
    
    # initialize w to random small numbers
    w = []
    for i in range(0, M):
        w.append(random.randint(1, 100) / 1000)
    w = np.asarray(w)
    
    # initialize a
    a = np.zeros(N)
    for i in range(0, N):
        intermed_value = np.dot(training_X[i, :] - input_X, training_X[i, :] - input_X)
        a[i] = -intermed_value / (2* bandwidth ** 2)
    a = np.exp(a)

    while iteration_numb < max_number_iterations and difference > threshold:
        delta = np.zeros(M)
        # for every training example we have:
        for k in range(0, N):
            # calculate y_k - x_k*w
            error = y[k] - np.dot(training_X[k,:], w)
            for i in range(0, M):
                delta[i] = delta[i] + a[k] * learning_rate * training_X[k, i] * error
        for i in range(0, M):
            w[i] = w[i] + delta[i];
            
        iteration_numb += 1
        difference = np.sqrt(delta.dot(delta))
    # We learned the weights, so just output the prediction
    return (w, np.dot(w, input_X))

# Obtain an analytical solution using normal equations
def loc_weighted_normal_eq(training_X, y, input_X, bandwidth = 0.5):
    # initialize a
    N = X.shape[0]
    a = np.zeros(N)
    for i in range(0, N):
        intermed_value = np.dot(training_X[i, :] - input_X, training_X[i, :] - input_X)
        a[i] = -intermed_value / (2* bandwidth ** 2)
    a = np.exp(a)
    a = np.diag(a)
    w = inv(training_X.transpose() @ a @ X) @ training_X.transpose() @ a @ y
    return (w, np.dot(w, input_X))

# Just a simple test
X = np.array([[5, 3, 2], [4, 2, 1], [3, 7, 0]])
y = np.array([1, 2, 3])
input_X = np.array([8, 2, 3])

(w, pred) = loc_weighted_regression(X, y, input_X, bandwidth = 3)
(analytical_w, analytical_pred) = loc_weighted_normal_eq(X, y, input_X, bandwidth = 3)
print(pred)
print(w)
print(analytical_pred)
print(analytical_w)

1.9999999999999627
[ 1.00000000e+00  2.03173538e-14 -2.00000000e+00]
2.000000000000006
[ 1.00000000e+00  3.44169138e-15 -2.00000000e+00]


Notice the importance of the bandwidth parameter. The higher the bandwidth, the more points we look at (around the given one). Note that the algorithm would fail to achieve a proper result, if the bandwidth is too low (in our example, *input_X* is quite far from training points). 

In [40]:
(w, pred) = loc_weighted_regression(X, y, input_X, bandwidth = .5)
(analytical_w, analytical_pred) = loc_weighted_normal_eq(X, y, input_X, bandwidth = .5)
print(pred)
print(w)
print(analytical_pred)
print(analytical_w)

0.21300608492972706
[0.01800059 0.01500035 0.01300023]
-2.9912676598669776
[ 0.31449722  2.15795708 -3.27438652]


<h2> Logistic Regression </h2>
Before, we were using ML models to predict a continuous variable. What if, instead, you need to classify datapoints according to just a few categories (e.g. input: picture -> output: like/dislike). This problem is called a *classification* problem, and one of the easiest models used for classification is logistic regression.

We will start with a binary classification (i.e. there are only two possible outcomes: 0 and 1). <br />
Instead of simply using $\textbf{w}^T\textbf{x}_i$ as an output of the model, we will use a so-called logistic (sigmoid) function of $\textbf{w}^T\textbf{x}_i$, which would be $\frac{1}{1 + e^{-\textbf{w}^T\textbf{x}_i}}$ <br />
The function that we aim to maximize (why is it a maximization problem now? This function is derived from probabilistic interpretaion \[ maximum likelihood \]. For those who are interested, please refer to CS229 Stanford Machine Learning course notes) is $\sum\limits_{i=1}^m{y_i \log (\textbf{w}^T\textbf{x}_i) + (1 - y_i)\log(1 - \textbf{w}^T\textbf{x}_i)}$ <br />
It is easy to derive the gradient ascent (note: we are using gradient ascent now, as we are maximizing the function) rule, which is: <br />
$w_i \leftarrow w_i + \alpha \sum\limits_{k=1}^n (y_k - sigm(\textbf{w}^T\textbf{x}_k)) x_{k,i}$ <br />
$\Delta w_i \leftarrow \alpha \sum\limits_{k=1}^n (y_k - sigm(\textbf{w}^T\textbf{x}_k)) x_{k,i}$ (for batch training) <br />

In [47]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, batch_size = None, max_numb_iterations = 10000, learning_rate = 0.1, threshold = 0):
    iteration_numb = 0
    difference = 1
    N = X.shape[0]
    M = X.shape[1]
    if batch_size == None:
        # Default to batch training
        batch_size = N
    
    full_data = np.column_stack([X, y])
    
    # initialize w to random small numbers
    w = []
    for i in range(0, M):
        w.append(random.randint(1, 100) / 1000)
    w = np.asarray(w)

    while iteration_numb < max_numb_iterations and difference > threshold:
        np.random.shuffle(full_data)
        
        mini_batches = [
            full_data[k : k + batch_size]
            for k in range(0, N, batch_size)]
        
        for mini_batch in mini_batches:
            delta = np.zeros(M)
            
            for training_example in mini_batch:
                # training example is of shape (M+1), with M features (x) and 1 output (y)
                error = training_example[-1] - sigmoid(np.dot(training_example[:-1], w))
                for i in range(0, M):
                    delta[i] = delta[i] + learning_rate * training_example[i] * error
            
            #Once this mini-batch is over, update weights:
            for i in range(0, M):
                w[i] = w[i] + delta[i]
                
            # Optional: if delta change is really small (smaller than threshold), break from this loop,
            # return the current weights
            difference = np.sqrt(delta.dot(delta))
            if(difference < threshold):
                break
            
        iteration_numb += 1
    return w

def predict(input_X, w):
    res = np.dot(input_X, w)
    if res >= .5:
        return 1
    else:
        return 0

How do we know if our simple logistic regression works satisfactorily? Let us check with the sklearn LogisticRegression model. Please note that sklearn's LogisticRegression automatically regularizes (we will cover regularization a bit later) and fits intercept point (or *bias term*, which I omitted for the purpose of simplicity). 

In [48]:
X = np.array([[-1, -2], [-2, -1], [6, 7], [2, 3]])
y = np.array([0, 0, 1, 1])
w = logistic_regression(X, y, batch_size = 4)
pred = predict(np.array([1.2, -1]), w)
print(w)
print(pred)

from sklearn.linear_model import LogisticRegression
# Set regularization parameter to a large number so that there is almost no regularization (to compare with our 
# unregularized model)
lr = LogisticRegression(fit_intercept = False, C = 1e22)
lr.fit(X, y)
print(lr.predict(np.array([1.2, -1]).reshape(1, -1)))
print(lr.coef_)

[3.00158627 3.07228656]
1
[1]
[[3.03910611 3.11125592]]


We see that the results are quite similar (learned weights differ only slightly, due to small number of iterations). <br />
Now let us introduce the concept of regularization into the Logistic Regression model. ( A good read on regularization: https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a )

We will solidify the logistic regression model by wrapping it into a Python class. We will keep only Stochastic Gradient Descent for the purpose of simplicity and we will also introduce the concept of vectorization as well (i.e. we will get rid of unnecessary for-loops and use numpy's ability to efficiently operate on arrays and matrices). For the illustration purposes, we will introduce a bias term (intercept term) in this model.

In [114]:
class MyLogisticRegression:
    
    '''
    fit_intercept - whether the bias term is included or not
    penalty - whether and what regularization to use (possible values: 'l1', 'l2' or 'none')
    C - regularization parameter
    threshold - tolerance level for stopping
    max_iter - maximum number of iterations before stopping
    lr - learning rate for SGD
    '''
    def __init__(self, fit_intercept = True, penalty = 'none', C = 1.0, threshold = 0, max_iter = 10000, lr = 0.1):
        self.fit_intercept = fit_intercept
        self.penalty = penalty
        self.C = C
        self.threshold = threshold
        self.max_iter = max_iter
        self.lr = lr
    
    def fit(self, X, y):
        iteration_numb = 0
        difference = 1
        N = X.shape[0]
        M = X.shape[1]
        
        if self.fit_intercept:
            M += 1
            X = np.hstack([np.ones((N, 1)), X])
            
        # initialize w to random small numbers
        self.w = []
        for i in range(0, M):
            self.w.append(random.randint(1, 100) / 1000)
        self.w = np.asarray(self.w)
            
        while iteration_numb < self.max_iter and difference > self.threshold:
            # Note that we got rid of all the loops. That is vectorization
            errors = y - 1 / (1 + np.exp(- X @ self.w))
            
            # Add in regularization.
            if self.penalty == 'l2':
                add_on =  - (1 / self.C) * self.w
            elif self.penalty == 'l1':
                add_on = - (1 / self.C) * np.sign(self.w)
            else:
                add_on = np.zeros(M)
                
            '''
            Note that sklearn algorithms regularize the intercept term, even though
            it is more common to exclude regularization for the intercept term
            
            In case we do not want to regularize the intercept term, use the following code snippet:
            if self.fit_intercept:
                add_on[0] = 0 # no regularization add-on for the intercept term
            ''' 
            delta = self.lr * (X.transpose() @ errors + add_on)
            self.w += delta
            
            # Optional: if delta change is really small (smaller than threshold), break from this loop,
            # return the current weights
            difference = np.sqrt(delta.dot(delta))
            if(difference < self.threshold):
                break
                
            iteration_numb += 1
                
    def predict(self, input_X):
        result = np.dot(input_X, self.w)
        if result >= .5:
            return 1
        else:
            return 0
        
    def coef_(self):
        return self.w
        
# Create a bigger random sample dataset
np.random.seed(12)
num_observations = 5000
x1 = np.random.multivariate_normal([0, 0], [[1, .75],[.75, 1]], num_observations)
x2 = np.random.multivariate_normal([1, 4], [[1, .75],[.75, 1]], num_observations)

X = np.vstack((x1, x2)).astype(np.float32)
y = np.hstack((np.zeros(num_observations),
                              np.ones(num_observations)))

lr = MyLogisticRegression(fit_intercept = True, lr = 5e-4, max_iter = 20000, threshold = 1e-10, penalty = 'l1', C = 3e15)

lr.fit(X, y)
w = lr.coef_()
print(w)

[-14.09050177  -5.05837918   8.28854616]


In [113]:
from sklearn.linear_model import LogisticRegression
# Set regularization parameter to a large number so that there is almost no regularization (to compare with our 
# unregularized model)
lr = LogisticRegression(fit_intercept = True, C = 3e15, max_iter = 1000, tol = 1e-10, penalty = 'l1')
lr.fit(X, y)
print(lr.coef_)
print(lr.intercept_)

[[-5.05901197  8.28958301]]
[-14.09229942]


We see that our logistic regression model works good enough. <br />
In this notebook, we've introduced a lot of simple models (Linear Regression, Locally Weighted Linear Regression and Logistic Regression). We also introduced a number of important concepts/algorithms (Gradient Descent: Batch, Stochastic and Mini-Batch; Bias/Intercept Term; L1/L2 regularization; Vectorization). 